In [84]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import iris

# Import data

In [85]:
## cube

fdir='/badc/srip/data/zonal/common_grid/era_interim/'
fpath='TEM_monthly_2000_01.nc'

# Fy: Want description primitive eqn: wavenumber "" (EPF_phi_pr)
standard_name='northward_eliassen_palm_flux_in_air'
con=iris.Constraint(standard_name)        
fy_fields=iris.load(fdir+fpath,con)
fy_fields=fy_fields.extract(iris.AttributeConstraint(wavenumber=""))
fy_fields=fy_fields.extract(iris.AttributeConstraint(description="Uses primitive equation"))
fy_fields=fy_fields[0]
if fy_fields.coords('air_pressure'): fy_fields.coord('air_pressure').rename('pressure') 
fy_fields = fy_fields[0]

In [86]:
## xarray

ds = xr.open_mfdataset(fdir+fpath)
Fphi = ds.EPF_phi_pr[0]

In [87]:
np.testing.assert_allclose(Fphi.values, fy_fields.data, rtol=1e-7)

In [88]:
# Constants
p0 = 1e3
a = 6.371e6

# Scale pressure coordinates

In [89]:
## cube

# define dimensions
pressure = fy_fields.coord('pressure').points
latitude = fy_fields.coord('latitude').points

# Convert from log pressure to pressure coordinates
plev_ratio = np.repeat(pressure/p0, latitude.size).reshape((pressure.size,latitude.size))
fy_fields = iris.analysis.maths.multiply(fy_fields, plev_ratio)

In [90]:
## xarray

# define dimensions
lat = ds.latitude.values
level = ds.pressure.values

# define and calculate ratio
p_ratio = np.repeat(level/p0, lat.size).reshape((level.size,lat.size))
Fphi = ds.EPF_phi_pr[0] * p_ratio                                       # [m3 s-2]

In [91]:
np.testing.assert_allclose(Fphi.values, fy_fields.data, rtol=1e-7)

# Calculate derivative

In [92]:
## cube

# Differentiate wrt φ
deriv1_cube=fy_fields.copy()

latsR = np.deg2rad(latitude)
# latsR= fy_fields.coord('latitude').points*2*np.pi/360	# in radians

cos_lats=np.cos(latsR)
Fcoslat=iris.analysis.maths.multiply(fy_fields,cos_lats)
deriv1_cube.data=np.gradient(Fcoslat.data,latsR,axis=1) # central difference except end points where forwards/backwards difference

In [93]:
## xarray

# convert lat to radians take np.cos and multiply by Fphi (inside derivative)
lat_rads = np.deg2rad(lat)
coslat = np.cos( lat_rads )  
F_coslat = Fphi * coslat

# calc derivative and convert lat dimension to radians
F_coslat['latitude'] = lat_rads
deriv1 = F_coslat.differentiate('latitude')                             # [m2 s-2]

In [94]:
np.testing.assert_allclose(deriv1.values, deriv1_cube.data, rtol=1e-7)

### Multiply by 1/acos(φ)

In [95]:
## cube

# Divide by a cos(φ)
div_Fphi_cube=iris.analysis.maths.divide(deriv1_cube,cos_lats)
div_Fphi_cube=iris.analysis.maths.divide(div_Fphi_cube,(a))

In [96]:
## xarray

# Divide by a cos(φ)
div_Fphi = deriv1 / (a * coslat) 

In [97]:
np.testing.assert_allclose(div_Fphi.values, div_Fphi_cube.data, rtol=1e-6)

# Create function

In [98]:
def div_Fphi(Fphi, apply_scaling=True):
    
    """
        Calculate divergence of northward component
        of EP flux, F_phi. Including an optional 
        scaling from log-pressure to pressure coords.
        
        ----------------------------------------------
        
        Input: xr.DataArray of epfy/Fy/Fphi 
        
        Out: xr.DataArray Div_Fphi
    """
    
    # define constants
    p0 = 1e3
    a = 6.371e6
    
    # define dimensions
    lat = ds.latitude.values
    level = ds.pressure.values
    
    if apply_scaling:
        # define and calculate ratio
        p_ratio = np.repeat(level/p0, lat.size).reshape((level.size,lat.size))
        Fphi = Fphi * p_ratio                                               # [m3 s-2]
    
    # convert lat to radians take np.cos and multiply by Fphi (inside derivative)
    lat_rads = np.deg2rad(lat)
    coslat = np.cos( lat_rads )  
    F_coslat = Fphi * coslat

    # calc derivative and convert lat dimension to radians
    F_coslat['latitude'] = lat_rads
    deriv1 = F_coslat.differentiate('latitude')                             # [m2 s-2]
    
    # Divide by a cos(φ)
    div_Fphi = deriv1 / (a * coslat)                                        # [m2 s-2]
    
    return div_Fphi

In [99]:
divFy = DivFphi(ds.EPF_phi_pr[0])

In [100]:
np.testing.assert_allclose(divFy.values, div_Fphi_cube.data, rtol=1e-6)